In [1]:
import cortex.polyutils
import sys
sys.path.append('/Users/mayra/PycharmProjects/CFM_RS_validation')
from helper_functions import *
from bold_preproc import *
from nilearn import image

In [13]:
## Create mean image
subjectid='sub-004'
space='fsnative'
#proj_PATH = '/Users/mayra/Library/CloudStorage/OneDrive-UMCG/Postdoc/CFLam'
task='ret'
source_directory = f'/Users/mayra/Library/CloudStorage/OneDrive-UMCG/Postdoc/coreg_test/func'
for run in {1,2,3,4}:
    bold_filename = f'{subjectid}_ses-1_task-{task}_run-{run}_bold.nii.gz'
    bold_file = op.join(source_directory, bold_filename)
    bold_img = nib.load(bold_file)
    # Step 2: Calculate the mean image
    mean_img = image.mean_img(bold_img)

    # Step 3: Display or save the mean image
    # Display the mean image
    mean_img.orthoview()
    # Optionally save the mean image to a file
    output_filename = f'mean_{subjectid}_ses-1_task-{task}_run-{run}_bold.nii.gz'
    output_file = op.join(source_directory, output_filename)
    mean_img.to_filename(output_file)


FileNotFoundError: No such file or no access: '/Users/mayra/Library/CloudStorage/OneDrive-UMCG/Postdoc/coreg_test/func/sub-004_ses-1_task-ret_run-1_bold.nii.gz'

In [36]:
bold_filename = f'mean_{subjectid}_ses-1_task-{task}_run-1_bold.nii.gz'
bold_file = op.join(source_directory, bold_filename)
bold_img1 = nib.load(bold_file)
bold_filename = f'mean_{subjectid}_ses-1_task-{task}_run-2_bold.nii.gz'
bold_file = op.join(source_directory, bold_filename)
bold_img2 = nib.load(bold_file)
bold_filename = f'mean_{subjectid}_ses-1_task-{task}_run-3_bold.nii.gz'
bold_file = op.join(source_directory, bold_filename)
bold_img3 = nib.load(bold_file)
bold_filename = f'mean_{subjectid}_ses-1_task-{task}_run-4_bold.nii.gz'
bold_file = op.join(source_directory, bold_filename)
bold_img4 = nib.load(bold_file)


Grand_mean_img = image.mean_img([bold_img1,bold_img2,bold_img3,bold_img4])
output_filename = f'Grmean_{subjectid}_ses-1_task-{task}_bold.nii.gz'
output_file = op.join(source_directory, output_filename)
Grand_mean_img.to_filename(output_file)

In [52]:
## Create freesurfer surface for ROIs delineation

# some basic python modules:
import os
opj = os.path.join
import numpy as np
import cortex
from nibabel.freesurfer.io import read_morph_data, write_morph_data # allows to read and write those curv files

project='CFLamUp'
denoising='nordic_sm4'
MAIN_PATH=f'/Users/mayra/Library/CloudStorage/OneDrive-UMCG/Postdoc/{project}/derivatives'
subjectid='sub-004'
depth='GM'
atlas='benson'
model_name='gauss_fitter'
# load your prf analysis results (if not already in a variable)
#prf_results = np.load(f'{MAIN_PATH}/pRFM/{subjectid}/ses-1/{denoising}/model-gauss_fitter_desc-prf_params.pkl', allow_pickle=True) # prf_results contains the matrix with the pRF results. It has the size: vertices (or voxels) x 6. The last column contains the r2 values. The first and second, the X and Y cartesian coordinates of the best fitting pRF position (the center of the Gaussian), respectively. From these coordinates we can extract both eccentricity and polar angle of the pRF! (for those interested, this page explains this conversion well: https://www.mathsisfun.com/polar-cartesian-coordinates.html.. basically just some refreshment of Pythagoras theorem etc.)
prf_results = np.load(f'{MAIN_PATH}/pRFM/{subjectid}/ses-1/{denoising}/model-{atlas}-{model_name}-{depth}_desc-prf_params.pkl', allow_pickle=True)
roi_verts=np.where(prf_results['rois_mask']==1)
prf_results=prf_results['model'].iterative_search_params

prf_params_vx=roi_verts[0]
angle_map = cortex.Vertex.empty(subjectid)
masked_polar_angle=np.zeros(angle_map.nverts)
masked_polar_angle[:]=50
masked_eccentricity=np.zeros(angle_map.nverts)
masked_eccentricity[:]=50
masked_r2=np.zeros(angle_map.nverts)
masked_r2[:]=0


# actually extract the prf measurements:
eccentricity = np.sqrt( prf_results[:,0]**2 + prf_results[:,1]**2 )
polar_angle = np.arctan2( prf_results[:,1], prf_results[:,0] )
r2 = prf_results[:,-1]

# some code that illustrates the masking:
r2_mask = r2 >0.1 # creates Boolean array with those vertices above the threshold
ecc_mask = eccentricity < 6
total_mask = r2_mask * ecc_mask # Boolean array where both above r squared and below maximum eccentricity
# note: you could include different things into your mask, such as minimal signal intensity, minimum eccentricity etc.
polar_angle[total_mask==False]=50
eccentricity[total_mask==False]=50
masked_polar_angle[roi_verts]=polar_angle
masked_eccentricity[roi_verts]=eccentricity
#r2[total_mask==False]=1
#r2[total_mask==True]=0
masked_r2[roi_verts]=r2



In [53]:
# freeview has separate curv files for left and right hemisphere, so if your prf analysis results are combined hemispheres we need to split them first:
fs_dir=f'{MAIN_PATH}/freesurfer/{subjectid}/surf/'
lh_c = read_morph_data(f'{fs_dir}/lh.curv')
#this is just to get the number of vertices for the left hemisphere. With that we can split the prf_results into the separate hemispheres. The curv files should be located in your freesurfer/subID/surfs folder.
lh_masked_pol = masked_polar_angle[:lh_c.shape[0]] # left hemisphere = all vertices up to the index
rh_masked_pol = masked_polar_angle[lh_c.shape[0]:] # all vertices from that vertex onwards belong to the right hemisphere
lh_masked_ecc = masked_eccentricity[:lh_c.shape[0]] # left hemisphere = all vertices up to the index
rh_masked_ecc = masked_eccentricity[lh_c.shape[0]:] # all vertices from that vertex onwards belong to the right hemisphere
lh_masked_r2 = masked_r2[:lh_c.shape[0]] # left hemisphere = all vertices up to the index
rh_masked_r2 = masked_r2[lh_c.shape[0]:] # all vertices from that vertex onwards belong to the right hemisphere


In [54]:
# now save the results as curv file:

write_morph_data(f'{fs_dir}/lh.masked_polext_{denoising}_{depth}', lh_masked_pol)
write_morph_data(f'{fs_dir}/rh.masked_polext_{denoising}_{depth}', rh_masked_pol)
write_morph_data(f'{fs_dir}/lh.masked_eccext_{denoising}_{depth}', lh_masked_ecc)
write_morph_data(f'{fs_dir}/rh.masked_eccext_{denoising}_{depth}', rh_masked_ecc)
# write_morph_data(f'{fs_dir}/lh.masked_badr2ext_{denoising}', lh_masked_r2)
# write_morph_data(f'{fs_dir}/rh.masked_badr2ext_{denoising}', rh_masked_r2)
# write_morph_data(f'{fs_dir}/lh.r2ext_{denoising}', lh_masked_r2)
# write_morph_data(f'{fs_dir}/rh.r2ext_{denoising}', rh_masked_r2)

# Done!


In [11]:
#In the terminal, freesurfer/surf:
freeview -f lh.inflated:overlay=lh.masked_badr2_nordic:overlay_custom=-3.14,255,0,0,-2.65,255,255,0,-2.09,0,128,0,-1.75,0,255,255,-1.05,0,0,255,-0.5,238,130,238,0,255,0,0,0.5,255,255,0,1.05,0,128,0,1.57,0,255,255,2.09,0,0,255,2.65,238,130,238,3.14,255,0,0 rh.inflated:overlay=rh.masked_badr2_nordic:overlay_custom=-3.14,255,0,0,-2.65,255,255,0,-2.09,0,128,0,-1.75,0,255,255,-1.05,0,0,255,-0.5,238,130,238,0,255,0,0,0.5,255,255,0,1.05,0,128,0,1.57,0,255,255,2.09,0,0,255,2.65,238,130,238,3.14,255,0,0

freeview -f surf/lh.inflated:overlay=lh.masked_polext_nordic_sm4:overlay_custom=colormaps/marco_pol:overlay_color='clearhigher':overlay=lh.masked_eccext_nordic_sm4:curvature_method='off':label=label/lh.manualV1.label:label=label/lh.manualV2.label:label=label/lh.manualV3.label:label=label/lh.manualV4.label:label=label/lh.manualLO1.label:label=label/lh.manualLO2.label

freeview -f surf/rh.inflated:overlay=rh.masked_polext_nordic_sm4:overlay_custom=colormaps/marco_pol:overlay_color='clearhigher':overlay=rh.masked_eccext_nordic_sm4:curvature_method='off':label=label/rh.manualV1.label:label=label/rh.manualV2.label:label=label/rh.manualV3.label:label=label/lh.manualV4.label:label=label/rh.manualLO1.label:label=label/rh.manualLO2.label:label=label/rh.manualout.label


SyntaxError: invalid syntax (3993000016.py, line 2)

In [55]:
freeview_cmd=f'freeview -f surf/lh.inflated:overlay=lh.masked_polext_{denoising}_{depth}:overlay_custom=../sub-001/colormaps/marco_pol:overlay_color=clearhigher:overlay=lh.masked_eccext_{denoising}_{depth}:curvature_method=off:label=label/lh.manualV1.label:label=label/lh.manualV2.label:label=label/lh.manualV3.label:label=label/lh.manualV4.label:label=label/lh.manualLO1.label:label=label/lh.manualLO2.label -f surf/rh.inflated:overlay=rh.masked_polext_{denoising}_{depth}:overlay_custom=../sub-001/colormaps/marco_pol:overlay_color=clearhigher:overlay=rh.masked_eccext_{denoising}_{depth}:curvature_method=off:label=label/rh.manualV1.label:label=label/rh.manualV2.label:label=label/rh.manualV3.label:label=label/rh.manualV4.label:label=label/rh.manualLO1.label:label=label/rh.manualLO2.label:label=label/rh.manualout.label'

print(freeview_cmd)

freeview -f surf/lh.inflated:overlay=lh.masked_polext_nordic_sm4_GM:overlay_custom=../sub-001/colormaps/marco_pol:overlay_color=clearhigher:overlay=lh.masked_eccext_nordic_sm4_GM:curvature_method=off:label=label/lh.manualV1.label:label=label/lh.manualV2.label:label=label/lh.manualV3.label:label=label/lh.manualV4.label:label=label/lh.manualLO1.label:label=label/lh.manualLO2.label -f surf/rh.inflated:overlay=rh.masked_polext_nordic_sm4_GM:overlay_custom=../sub-001/colormaps/marco_pol:overlay_color=clearhigher:overlay=rh.masked_eccext_nordic_sm4_GM:curvature_method=off:label=label/rh.manualV1.label:label=label/rh.manualV2.label:label=label/rh.manualV3.label:label=label/rh.manualV4.label:label=label/rh.manualLO1.label:label=label/rh.manualLO2.label:label=label/rh.manualout.label


In [1]:
## Read and edit freesurfer label files

with open(fname) as fid:
    lines0 = fid.readlines()
    lines1 = [[float(xx.strip()) for xx in x.split(' ') if xx.strip()] for x in lines0[2:]]
    vals = np.array(lines1)
    vals[:,4]=1.0

NameError: name 'fname' is not defined

In [11]:
subjectid='sub-003'
import numpy as np
rois_list = np.array([['V1', 'V2', 'V3','V4','LO1', 'LO2', 'out'], [1, 2, 3,4,7, 8,0]])

for hemi in {'lh', 'rh'}:
    for roi in range(rois_list[0].__len__()):
        fname=f'/Users/mayra/Library/CloudStorage/OneDrive-UMCG/Postdoc/CFLamUp/derivatives/freesurfer/{subjectid}/label/{hemi}.manual{rois_list[0][roi]}.label'
        outname=f'/Users/mayra/Library/CloudStorage/OneDrive-UMCG/Postdoc/CFLamUp/derivatives/freesurfer/{subjectid}/label/{hemi}.manual{rois_list[0][roi]}edit.label'
        with open(fname, 'r') as f_in, open(outname, 'w') as f_out:
            counter=0
            for line in f_in:
                counter=counter+1
                if counter>2:
                    output_line = line
                    editline=output_line.split(' ')
                    editline[7] =f'{rois_list[1][roi]}\n'
                    output_line=' '.join(editline)
                else:
                    output_line = line
                f_out.write(output_line)
                print(output_line)

#!ascii label  , from subject  vox2ras=TkReg

2475

8270  15.296  -82.211  2.787 1

7674  15.547  -82.735  2.617 1

7681  14.888  -82.490  2.358 1

8271  14.715  -81.954  2.860 1

8733  15.332  -81.778  3.279 1

8259  15.692  -82.385  3.311 1

7874  13.908  -82.337  2.535 1

8272  13.877  -81.647  3.075 1

8738  14.698  -81.311  3.397 1

8273  13.220  -81.954  2.880 1

8968  13.099  -81.329  3.143 1

137196  13.877  -80.909  3.411 1

7682  12.898  -82.487  2.568 1

8744  12.531  -81.821  2.821 1

149393  15.474  -81.161  3.803 1

137806  15.826  -81.876  3.915 1

9390  14.806  -80.365  3.819 1

7677  12.054  -82.442  2.522 1

8969  12.422  -81.035  2.897 1

8276  11.877  -81.711  2.529 1

144383  13.179  -80.539  3.232 1

146989  11.772  -81.048  2.423 1

138458  12.106  -80.395  2.592 1

9398  12.659  -80.016  2.938 1

8967  15.693  -80.524  4.284 1

10455  14.825  -79.484  4.299 1

9968  15.406  -79.895  4.370 1

9394  13.867  -80.192  3.519 1

9971  14.133  -79.561  3.844 1

10463  

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [34]:
#in the terminal:
mri_mergelabels -i rh.manualV1edit.label -i rh.manualV2edit.label -i rh.manualV3edit.label -i rh.manualLO1edit.label -i rh.manualLO2edit.label -i rh.manualoutedit.label -o rh.manualdelin.label

mri_mergelabels -i lh.manualV1edit.label -i lh.manualV2edit.label -i lh.manualV3edit.label -i lh.manualLO1edit.label -i lh.manualLO2edit.label -i lh.manualoutedit.label -o lh.manualdelin.label

'V3'

43